### Baseline 1: Extractive timeline summarization

1) **Retrieval**: 
    - Semantic
    - BM25
    - BM25 + Semantic

2) **Event grouping**: near-duplicate clustering (cosine > 0.97)

3) **Summary**: extractive по репрезентативным источникам

4) **Resonance**: количество упоминаний/каналов

In [1]:
! pip install scikit-learn rank_bm25 sentence_transformers

Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
! pip install accelerate

Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [1]:
import pandas as pd

df = pd.read_csv("../data/cleaned_news_exp.csv")[["message_id", "id_channel", "message", "date", "topic", "viral_final"]]
df.head()

,message_id,id_channel,message,date,topic,viral_final
0,0007e2f8-787d-404f-91ff-e2582096a4a7,18,Сербия согласна поддержать санкции Евросоюза п...,2025-07-26 07:01:09,Санкции и геополитика,0.703622
1,000884a5-8291-4ec1-805f-ac131112aaf7,6,Китайский рынок акций упал сильнее всего с апр...,2025-09-04 10:16:56,Рынки капитала,0.680369
2,000b0331-92a9-4eb4-9f58-d00811257758,18,Министерство труда США отменило рекомендации 2...,2025-05-29 04:05:09,Государственная экономическая политика,0.589152
3,000b8df7-d902-41eb-b668-900614902f0a,6,Чистая прибыль Московской биржи по МСФО во вто...,2025-08-26 11:40:55,Корпоративные финансы,0.461692
4,0011adea-7a98-4dcc-b753-905597b42788,4,"США хотят получить нефть и «всё, что угодно» о...",2025-02-22 20:57:18,Сырьевые рынки,0.504362


In [2]:
df[df["message_id"] == "588a6960-aa5b-406b-8417-fdad3ccde1e9"]

,message_id,id_channel,message,date,topic,viral_final
6460,588a6960-aa5b-406b-8417-fdad3ccde1e9,5,Кофе в России может подорожать на 50% уже в эт...,2025-04-07 13:32:23,Сырьевые рынки,0.716038


In [5]:
import re
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple

from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
def ensure_datetime(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    d = df.copy()
    d[col] = pd.to_datetime(d[col], utc=True, errors="coerce")
    d = d.dropna(subset=[col])
    d["date_day"] = d[col].dt.floor("D")
    return d

def normalize_text(s: str) -> str:
    s = s or ""
    s = re.sub(r"\s+", " ", s).strip()
    return s

def l2_normalize(x: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True)
    return x / np.maximum(n, eps)

def cosine_scores(query_vec: np.ndarray, doc_mat_l2: np.ndarray) -> np.ndarray:
    q = l2_normalize(query_vec.reshape(1, -1))[0]
    return doc_mat_l2 @ q

def topk_from_scores(scores: np.ndarray, k: int) -> Tuple[np.ndarray, np.ndarray]:
    k = min(k, len(scores))
    if k == len(scores):
        idx = np.argsort(-scores)
    else:
        idx = np.argpartition(-scores, k)[:k]
        idx = idx[np.argsort(-scores[idx])]
    return idx, scores[idx]

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
from pathlib import Path

_E5_MODEL_NAME = "intfloat/multilingual-e5-large"
_e5_model = None

def _get_e5():
    global _e5_model
    if _e5_model is None:
        _e5_model = SentenceTransformer(_E5_MODEL_NAME)
    return _e5_model

class Encoder:
    def __init__(self, default_prefix: str = "query: "):
        self.default_prefix = default_prefix

    def _ensure_prefix(self, t: str) -> str:
        t = t or ""
        if t.startswith("query:") or t.startswith("passage:"):
            return t
        return (self.default_prefix + t) if self.default_prefix else t

    def encode_texts(self, texts):
        mod = _get_e5()
        texts = [self._ensure_prefix(x) for x in texts]
        vecs = mod.encode(texts, normalize_embeddings=True, show_progress_bar=False)
        return vecs.astype(np.float32)

def encode_query(query_text: str) -> np.ndarray:
    enc = Encoder(default_prefix="query: ")
    return enc.encode_texts([query_text])[0]


def load_and_normalize_emb(path: Path) -> np.ndarray:
    E = np.load(path, mmap_mode="r")
    X = E.astype(np.float32)
    X /= (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)
    return X

E_docs = load_and_normalize_emb(Path("embeddings/emb_e5_large_fp16.npy"))
encoder = Encoder(default_prefix="query: ")


## Semantic retrieve

In [8]:
from typing import Tuple, Optional
import pandas as pd
import numpy as np

def semantic_retrieve(
    df: pd.DataFrame,
    E_docs_l2: np.ndarray,
    query: str,
    encoder: Encoder,
    k: int = 200,
    window_days: Optional[int] = None,
    end_date: Optional[pd.Timestamp] = None,
) -> pd.DataFrame:
    
    d = ensure_datetime(df, "date")
    if end_date is None:
        end_date = d["date_day"].max()
    end_date = pd.to_datetime(end_date, utc=True)
    if window_days is not None:
        start_date = end_date - pd.Timedelta(days=window_days - 1)
        mask = (d["date_day"] >= start_date) & (d["date_day"] <= end_date)
        idx_pool = np.where(mask.values)[0]
    else:
        idx_pool = np.arange(len(d))

    qv = encoder.encode_texts([f"query: {query}"])[0]
    scores = cosine_scores(qv, E_docs_l2[idx_pool])
    local_idx, local_scores = topk_from_scores(scores, k=min(k, len(idx_pool)))
    glob_idx = idx_pool[local_idx]

    out = d.iloc[glob_idx].copy()
    out["sim"] = local_scores
    return out.sort_values("sim", ascending=False).reset_index(drop=True)

In [9]:
cand = semantic_retrieve(
    df=df,
    E_docs_l2=E_docs,
    query="кофе дорожает",
    encoder=encoder,
    k=200,
    window_days=None,
)

cand[["date_day","id_channel","message_id","sim","message", "topic"]].head(20)


,date_day,id_channel,message_id,sim,message,topic
0,2025-08-01 00:00:00+00:00,5,8fbc1ae5-130c-4773-ba34-5e004af5861e,0.892354,В России ожидается новое подорожание кофе — на...,Сырьевые рынки
1,2025-07-10 00:00:00+00:00,4,5498bb88-6944-4a0b-9a19-324fc0b16c2e,0.890690,Кофе подорожает на 40% в этом году — прогноз э...,Сырьевые рынки
2,2025-03-04 00:00:00+00:00,4,52e1439a-fe85-4305-a407-0bfc2b873efa,0.888718,Кофе рекордно подорожает на 30-40% уже к концу...,Сырьевые рынки
3,2025-03-24 00:00:00+00:00,4,4f13a7ff-e768-43ef-a71b-e429f3dada1b,0.888538,Кофе может резко подорожать до 40% уже в этом ...,Сырьевые рынки
4,2025-01-29 00:00:00+00:00,3,0269bcc9-dcec-4317-af62-cf179f2bc854,0.884923,Биржевые цены на кофе сорта арабика вновь бьют...,Сырьевые рынки
5,2025-01-09 00:00:00+00:00,18,f7212d03-4fc9-4c70-b58c-bfb3336ff4ca,0.880584,Цены на кофе и шоколад в этом году взлетят на ...,Сырьевые рынки
6,2025-01-30 00:00:00+00:00,6,947db7a2-0784-4980-8ef5-8b2f7d2bca9c,0.875838,Крупнейшие поставщики собираются увеличить отп...,Сырьевые рынки
7,2025-01-30 00:00:00+00:00,5,746d754f-a71f-4717-a59c-a0601ac0fe16,0.874671,Цены на кофе в России подскочат ещё на 20% уже...,Сырьевые рынки
8,2025-04-07 00:00:00+00:00,5,588a6960-aa5b-406b-8417-fdad3ccde1e9,0.872903,Кофе в России может подорожать на 50% уже в эт...,Сырьевые рынки
9,2025-06-16 00:00:00+00:00,3,d009f7ce-5386-4530-a117-f8c70c23288f,0.871752,Кофе и какао в мире могут подорожать из-за тор...,Сырьевые рынки


## BM25 retrieve

In [10]:
from typing import Optional
import numpy as np
import pandas as pd
import re
from rank_bm25 import BM25Okapi

def _tok(s: str):
    return re.findall(r"[A-Za-zА-Яа-яЁё0-9_]+", (s or "").lower())

def build_bm25_index(df: pd.DataFrame, text_col: str = "message"):
    texts = df[text_col].fillna("").astype(str).tolist()
    tokenized = [_tok(t) for t in texts]
    bm25 = BM25Okapi(tokenized)
    return bm25

def bm25_retrieve(
    df: pd.DataFrame,
    bm25,
    query: str,
    k: int = 200,
    window_days: Optional[int] = None,
    end_date: Optional[pd.Timestamp] = None,
) -> pd.DataFrame:
    d = ensure_datetime(df, "date")
    if "channel" not in d.columns and "id_channel" in d.columns:
        d = d.rename(columns={"id_channel": "channel"})
    if end_date is None:
        end_date = d["date_day"].max()
    end_date = pd.to_datetime(end_date, utc=True)

    if window_days is not None:
        start_date = end_date - pd.Timedelta(days=window_days - 1)
        mask = (d["date_day"] >= start_date) & (d["date_day"] <= end_date)
        idx_pool = np.where(mask.values)[0]
    else:
        idx_pool = np.arange(len(d))

    q = _tok(query)
    scores_full = np.asarray(bm25.get_scores(q), dtype=np.float32)
    scores = scores_full[idx_pool]
    local_idx, local_scores = topk_from_scores(scores, k=min(k, len(idx_pool)))
    glob_idx = idx_pool[local_idx]

    out = d.iloc[glob_idx].copy()
    out["bm25"] = local_scores
    return out.sort_values("bm25", ascending=False).reset_index(drop=True)

In [11]:
bm25 = build_bm25_index(df, text_col="message")
cand_bm25 = bm25_retrieve(df, bm25, "кофе дорожает", k=200, window_days=None)
cand_bm25[["date_day","channel","message_id","bm25","message", "topic"]].head(20)

,date_day,channel,message_id,bm25,message,topic
0,2025-02-14 00:00:00+00:00,5,2d0ccb5e-3519-4467-834c-ac9259ff454f,11.547793,Доллар дорожает после решения ЦБ сохранить клю...,Валютный рынок
1,2025-03-07 00:00:00+00:00,6,080517ab-04a4-43bb-aa4f-94e4f530ea39,10.647270,Мировая торговля кофе практически остановилась...,Международная торговля
2,2025-03-24 00:00:00+00:00,4,4f13a7ff-e768-43ef-a71b-e429f3dada1b,10.432204,Кофе может резко подорожать до 40% уже в этом ...,Сырьевые рынки
3,2025-03-07 00:00:00+00:00,1,6975c760-98c8-4d1f-b7ae-0575040dc163,10.403980,Мировая торговля кофе практически остановилась...,Сырьевые рынки
4,2025-07-09 00:00:00+00:00,4,bc138891-3c81-47d9-9f19-f16ba756a293,10.349295,"Китайский автомобиль дорожает в России в 2,5 р...",Государственная экономическая политика
5,2025-08-19 00:00:00+00:00,4,0245935c-029c-4c1a-8aa1-ab5377d7ec9d,10.308459,"Бутербродекс подешевел на 2,1 пункта впервые з...",Макроэкономика
6,2025-07-10 00:00:00+00:00,4,5498bb88-6944-4a0b-9a19-324fc0b16c2e,10.187615,Кофе подорожает на 40% в этом году — прогноз э...,Сырьевые рынки
7,2025-05-14 00:00:00+00:00,6,0d7746ee-7c87-4ad1-aa6e-9fe858ddb69f,10.128250,Торги фьючерсами на кофе на срочном рынке Мосб...,Сырьевые рынки
8,2025-05-14 00:00:00+00:00,1,2f55600b-5ffe-48f4-9ce1-70bc21205f84,9.836158,Московская биржа с 20 мая запустит торги фьюче...,Рынки капитала
9,2025-04-19 00:00:00+00:00,3,ce0ff05d-0916-4fd0-b197-b7b9e5fcc923,9.733049,Колумбийский эксперт и бариста Себастьян Сулуа...,Сырьевые рынки


## Hybrid

In [24]:
from typing import Optional
import numpy as np
import pandas as pd

def rrf_fuse_weighted(sem_idx, bm_idx, k_rrf=60, w_sem=1.0, w_bm=1.0):
    scores = {}
    for w, rank in ((w_sem, sem_idx), (w_bm, bm_idx)):
        for i, doc_id in enumerate(rank):
            scores[doc_id] = scores.get(doc_id, 0.0) + w * (1.0 / (k_rrf + i + 1))
    ids = np.array(sorted(scores.keys(), key=lambda x: -scores[x]), dtype=int)
    vals = np.array([scores[i] for i in ids], dtype=np.float32)
    return ids, vals


def hybrid_retrieve(
    df: pd.DataFrame,
    E_docs_l2: np.ndarray,
    bm25,
    query: str,
    encoder: Encoder,
    k: int = 200,
    topN_each: int = 1000,
    k_rrf: int = 60,
    w_sem: float = 1.0,
    w_bm: float = 1.0,
    window_days: Optional[int] = None,
    end_date: Optional[pd.Timestamp] = None,
) -> pd.DataFrame:
    d = ensure_datetime(df, "date")
    if "channel" not in d.columns and "id_channel" in d.columns:
        d = d.rename(columns={"id_channel": "channel"})
    if end_date is None:
        end_date = d["date_day"].max()
    end_date = pd.to_datetime(end_date, utc=True)

    if window_days is not None:
        start_date = end_date - pd.Timedelta(days=window_days - 1)
        mask = (d["date_day"] >= start_date) & (d["date_day"] <= end_date)
        idx_pool = np.where(mask.values)[0]
    else:
        idx_pool = np.arange(len(d))

    qv = encoder.encode_texts([f"query: {query}"])[0]
    sem_scores = cosine_scores(qv, E_docs_l2[idx_pool])
    sem_local, _ = topk_from_scores(sem_scores, k=min(topN_each, len(idx_pool)))
    sem_rank = idx_pool[sem_local].tolist()

    q = _tok(query)
    bm_scores_full = np.asarray(bm25.get_scores(q), dtype=np.float32)
    bm_scores = bm_scores_full[idx_pool]
    bm_local, _ = topk_from_scores(bm_scores, k=min(topN_each, len(idx_pool)))
    bm_rank = idx_pool[bm_local].tolist()

    fused_ids, fused_scores = rrf_fuse_weighted(
        sem_rank, bm_rank, k_rrf=k_rrf, w_sem=w_sem, w_bm=w_bm
    )
    fused_ids = fused_ids[:min(k, len(fused_ids))]
    fused_scores = fused_scores[:len(fused_ids)]

    out = d.iloc[fused_ids].copy()
    out["rrf"] = fused_scores
    out["sem_sim"] = cosine_scores(qv, E_docs_l2[fused_ids])
    out["bm25"] = bm_scores_full[fused_ids]
    return out.sort_values("rrf", ascending=False).reset_index(drop=True)



In [29]:
cand_h = hybrid_retrieve(
    df=df,
    E_docs_l2=E_docs,
    bm25=bm25,
    query="инфляция в России ускорилась",
    encoder=encoder,
    k=200,
    topN_each=2000,
    k_rrf=10,
    w_sem=1.2,
    w_bm=0.5,
    window_days=None,
)


cand_h[["date_day","channel","message_id","rrf","sem_sim","bm25","message", "topic"]].head(20)

,date_day,channel,message_id,rrf,sem_sim,bm25,message,topic
0,2025-04-02 00:00:00+00:00,4,f0e1003c-42db-4e3c-b284-fa59eb09e26e,0.130769,0.894102,20.881771,Инфляция в РФ за последнюю неделю марта ускори...,Макроэкономика
1,2025-03-12 00:00:00+00:00,4,d2da9c8e-8843-4401-a14c-d36175a0e9c5,0.125455,0.892622,21.423134,Годовая инфляция в России в феврале 2025 года ...,Макроэкономика
2,2025-03-27 00:00:00+00:00,4,ab430d31-e452-4ef1-b4c3-399039932851,0.117565,0.906718,10.609774,В России снова ускоряется инфляция. С 18 по 24...,Макроэкономика
3,2025-02-12 00:00:00+00:00,4,8859f729-dca1-4392-ba71-01755b91755f,0.110714,0.893529,17.652006,"Годовая инфляция в России ускорилась с 9,92 до...",Макроэкономика
4,2025-02-26 00:00:00+00:00,5,60444759-aa64-4470-b646-e4a661f5a6af,0.106944,0.901628,10.244429,Росстат зафиксировал ускорение инфляции в Росс...,Макроэкономика
5,2025-04-02 00:00:00+00:00,2,ef844539-bb86-48d8-9365-9a165a9a5236,0.090476,0.879926,17.122869,#Макро\n⚡️ Годовая инфляция на 31 марта ускори...,Макроэкономика
6,2025-01-29 00:00:00+00:00,2,42c13da1-6d57-4ec8-aad8-27874c529d6e,0.089474,0.876022,18.216091,ГОДОВАЯ ИНФЛЯЦИЯ В РФ НА 27 ЯНВАРЯ УСКОРИЛАСЬ ...,Макроэкономика
7,2025-02-26 00:00:00+00:00,2,ab384162-2e13-4f98-9f6f-fb60f1d1a489,0.087778,0.875920,18.216091,ГОДОВАЯ ИНФЛЯЦИЯ В РФ НА 24 ФЕВРАЛЯ УСКОРИЛАСЬ...,Макроэкономика
8,2025-02-26 00:00:00+00:00,5,02e9e579-71c8-47e9-a097-337dd62a1f25,0.082042,0.887457,10.409454,Инфляция в России продолжает лететь в космос. ...,Макроэкономика
9,2025-01-22 00:00:00+00:00,2,04da9cd9-78ce-4d15-8e52-41f55396ecb3,0.074901,0.869888,17.122869,#Макро\n⚡️ Годовая инфляция на 20 января ускор...,Макроэкономика


## Посравниваем

In [14]:
import json, re, torch
from transformers import AutoTokenizer, AutoModelForCausalLM

JUDGE_MODEL = "Qwen/Qwen2.5-7B-Instruct"

judge_tokenizer = AutoTokenizer.from_pretrained(JUDGE_MODEL, trust_remote_code=True)
judge_model = AutoModelForCausalLM.from_pretrained(
    JUDGE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)
judge_model.eval()

def _extract_first_json_obj(text: str):
    text = (text or "").strip()
    if text.startswith("{") and text.endswith("}"):
        try:
            return json.loads(text)
        except Exception:
            pass
    for m in re.finditer(r"\{.*?\}", text, flags=re.DOTALL):
        cand = m.group(0)
        try:
            return json.loads(cand)
        except Exception:
            continue
    return None


@torch.inference_mode()
def judge_pairs_batched(pairs, batch_size=32, max_new_tokens=40):
    out_scores = []
    for i in range(0, len(pairs), batch_size):
        chunk = pairs[i:i+batch_size]

        prompts = []
        for q, sn in chunk:
            user = f"Запрос:\n{q}\n\nДокумент:\n{sn}\n"
            messages = [
                {"role": "system", "content": JUDGE_SYSTEM},
                {"role": "user", "content": user},
            ]
            prompts.append(
                judge_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            )

        enc = judge_tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
        ).to(judge_model.device)

        gen_ids = judge_model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            eos_token_id=judge_tokenizer.eos_token_id,
            pad_token_id=judge_tokenizer.eos_token_id,
        )

        for b in range(len(chunk)):
            prompt_len = int(enc["attention_mask"][b].sum().item())
            gen_txt = judge_tokenizer.decode(gen_ids[b][prompt_len:], skip_special_tokens=True)
            data = _extract_first_json_obj(gen_txt)
            rel = int(data["relevance"]) if isinstance(data, dict) and data.get("relevance") in (0,1,2) else 0
            out_scores.append(rel)

    return out_scores

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [15]:
JUDGE_SYSTEM = """Ты строгий эксперт по информационному поиску.

Оцени релевантность документа запросу, используя ТОЛЬКО текст документа.

Шкала:
2 — документ прямо отвечает запросу / описывает то же событие или факт.
1 — документ тематически близок, но не отвечает напрямую.
0 — нерелевантен.

Если в документе недостаточно информации, ставь 0 или 1 (не 2).
Верни только JSON: {"relevance": 0|1|2}
"""

In [49]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

K = 50
SEED = 67
N_MANUAL = 20
N_AUTO = 30

manual_queries = list(dict.fromkeys([
    "рост цен на кофе",
    "ключевая ставка цб решение",
    "повышение ключевой ставки",
    "снижение ключевой ставки",
    "инфляция ускорилась",
    "инфляция замедлилась",
    "индекс потребительских цен ипц",
    "курс рубля к доллару",
    "курс евро к рублю",
    "ослабление рубля причины",
    "нефть brent падение",
    "нефть brent рост",
    "цена нефти brent прогноз",
    "газ в европе цены",
    "газ в европе прогноз",
    "мосбиржа индекс imoex рост",
    "мосбиржа индекс imoex падение",
    "санкции влияние на экономику",
    "санкции против россии новые",
    "ограничения на экспорт влияние",
]))

def clean_auto_query(t: str) -> str:
    t = re.sub(r"#\w+", " ", t)
    t = re.sub(r"[⚡️📈📉🇷🇺✅❗️🔥⬛ ⬜ ⚫ ⚪🔹]+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t[:180]

def snippet(t: str, n: int = 800) -> str:
    t = re.sub(r"\s+", " ", (t or "").strip())
    return t[:n]

def first_sentence(text: str) -> str:
    t = re.sub(r"\s+", " ", (text or "").strip())
    parts = re.split(r"[.!?…]", t, maxsplit=1)
    s = (parts[0] if parts else t).strip()
    return (s if s else t)[:140]

def dcg(rels):
    rels = np.array(rels, dtype=float)
    denom = np.log2(np.arange(2, len(rels) + 2))
    return float(np.sum((2**rels - 1) / denom))

def ndcg_at_k(rels, k=50):
    rels = list(rels)[:k]
    ideal = sorted(rels, reverse=True)
    denom = dcg(ideal)
    return 0.0 if denom == 0 else dcg(rels) / denom

def precision_at_k(rels, k=50, thr=1):
    rels = np.array(list(rels)[:k])
    return float(np.mean(rels >= thr)) if len(rels) else 0.0

def mrr_at_k(rels, k=50, thr=2):
    rels = list(rels)[:k]
    for i, r in enumerate(rels, start=1):
        if r >= thr:
            return 1.0 / i
    return 0.0

rng = np.random.default_rng(SEED)
pool = df["message"].fillna("").astype(str)
pool = pool[pool.str.len() >= 40]
auto_texts = pool.sample(n=min(N_AUTO, len(pool)), random_state=SEED).tolist()
auto_queries = [clean_auto_query(first_sentence(x)) for x in auto_texts]
auto_queries = [q for q in auto_queries if len(q) >= 25]

queries = manual_queries[:N_MANUAL] + auto_queries
queries = queries[: (N_MANUAL + N_AUTO)]
len(queries), queries


(50,
 ['рост цен на кофе',
  'ключевая ставка цб решение',
  'повышение ключевой ставки',
  'снижение ключевой ставки',
  'инфляция ускорилась',
  'инфляция замедлилась',
  'индекс потребительских цен ипц',
  'курс рубля к доллару',
  'курс евро к рублю',
  'ослабление рубля причины',
  'нефть brent падение',
  'нефть brent рост',
  'цена нефти brent прогноз',
  'газ в европе цены',
  'газ в европе прогноз',
  'мосбиржа индекс imoex рост',
  'мосбиржа индекс imoex падение',
  'санкции влияние на экономику',
  'санкции против россии новые',
  'ограничения на экспорт влияние',
  'Погрузка на сети РЖД в июле снизилась на 5',
  'В последние годы социально-экономическое неравенство в мире продолжает стремительно расти',
  'Уолл-стрит завершила работу 3 апреля с большими потерями из-за опасений, что таможенные тарифы Трампа спровоцируют рецессию, передает Reuter',
  'Госзакупки программно-аппаратных комплексов с ускорителями для ИИ в 2024 году выросли на 150% в денежном выражении, до 2,4 млр

In [53]:
def _take_top_docs(cand: pd.DataFrame, k: int):
    cand = cand.copy()
    if "channel" not in cand.columns and "id_channel" in cand.columns:
        cand = cand.rename(columns={"id_channel": "channel"})
    cand = cand.head(k)
    ids = cand["message_id"].astype(str).tolist()
    docs = [snippet(x) for x in cand["message"].fillna("").astype(str).tolist()]
    return ids, docs

def eval_three_retrievers(
    df: pd.DataFrame,
    E_docs: np.ndarray,
    encoder,
    bm25,
    queries: list[str],
    k: int = 50,
    window_days=None,
    topN_each: int = 1000,
    k_rrf: int = 60,
    w_sem: float = 1.0,
    w_bm: float = 1.0,
    batch_size: int = 32,
):
    metric_rows = []
    pair_rows = []

    for q in tqdm(queries, desc="Queries"):
        cand_sem = semantic_retrieve(df, E_docs, q, encoder, k=k, window_days=window_days)
        cand_bm  = bm25_retrieve(df, bm25, q, k=k, window_days=window_days)
        cand_hyb = hybrid_retrieve(
            df, E_docs, bm25, q, encoder,
            k=k,
            topN_each=topN_each,
            k_rrf=k_rrf,
            w_sem=w_sem,
            w_bm=w_bm,
            window_days=window_days
        )

        packs = [
            ("semantic", cand_sem),
            ("bm25", cand_bm),
            ("hybrid", cand_hyb),
        ]

        for method, cand in packs:
            ids, docs = _take_top_docs(cand, k)
            rels = judge_pairs_batched([(q, d) for d in docs], batch_size=batch_size)

            metric_rows.append({
                "query": q,
                "method": method,
                "ndcg@50": ndcg_at_k(rels, k),
                "mrr@50(rel=2)": mrr_at_k(rels, k, thr=2),
                "p@50(rel=2)": precision_at_k(rels, k, thr=2),
                "p@50(rel>=1)": precision_at_k(rels, k, thr=1),
                "n_docs": len(rels),
            })

            for r, (mid, rel) in enumerate(zip(ids, rels), start=1):
                pair_rows.append({
                    "query": q,
                    "method": method,
                    "rank": r,
                    "message_id": mid,
                    "relevance": int(rel),
                })

    return pd.DataFrame(metric_rows), pd.DataFrame(pair_rows)


metrics_df, pairs_df = eval_three_retrievers(
    df=df,
    E_docs=E_docs,
    encoder=encoder,
    bm25=bm25,
    queries=queries,
    k=50,
    window_days=None,
    topN_each=2000,
    k_rrf=60,
    w_sem=1.0,
    w_bm=0.25,
    batch_size=32,
)


Queries: 100%|██████████| 50/50 [08:14<00:00,  9.89s/it]


In [54]:
summary = metrics_df.groupby("method", as_index=False)[["ndcg@50","mrr@50(rel=2)","p@50(rel=2)","p@50(rel>=1)"]].mean()
summary

,method,ndcg@50,mrr@50(rel=2),p@50(rel=2),p@50(rel>=1)
0,bm25,0.811421,0.721281,0.1580,0.3640
1,hybrid,0.858332,0.772857,0.2100,0.4956
2,semantic,0.859573,0.777857,0.2084,0.4936


In [55]:
pivot_ndcg = metrics_df.pivot(index="query", columns="method", values="ndcg@50")
pivot_mrr  = metrics_df.pivot(index="query", columns="method", values="mrr@50(rel=2)")
pivot_p2   = metrics_df.pivot(index="query", columns="method", values="p@50(rel=2)")

win_ndcg = {
    "hybrid>semantic": float((pivot_ndcg["hybrid"] > pivot_ndcg["semantic"]).mean()),
    "hybrid>bm25": float((pivot_ndcg["hybrid"] > pivot_ndcg["bm25"]).mean()),
    "semantic>bm25": float((pivot_ndcg["semantic"] > pivot_ndcg["bm25"]).mean()),
}
win_mrr = {
    "hybrid>semantic": float((pivot_mrr["hybrid"] > pivot_mrr["semantic"]).mean()),
    "hybrid>bm25": float((pivot_mrr["hybrid"] > pivot_mrr["bm25"]).mean()),
    "semantic>bm25": float((pivot_mrr["semantic"] > pivot_mrr["bm25"]).mean()),
}
win_p2 = {
    "hybrid>semantic": float((pivot_p2["hybrid"] > pivot_p2["semantic"]).mean()),
    "hybrid>bm25": float((pivot_p2["hybrid"] > pivot_p2["bm25"]).mean()),
    "semantic>bm25": float((pivot_p2["semantic"] > pivot_p2["bm25"]).mean()),
}

win_ndcg, win_mrr, win_p2

({'hybrid>semantic': 0.34, 'hybrid>bm25': 0.68, 'semantic>bm25': 0.66},
 {'hybrid>semantic': 0.06, 'hybrid>bm25': 0.18, 'semantic>bm25': 0.2},
 {'hybrid>semantic': 0.14, 'hybrid>bm25': 0.44, 'semantic>bm25': 0.42})